!pip install implicit

!pip install rectools

In [1]:
import os

import numpy as np
import pandas as pd

from implicit.als import AlternatingLeastSquares

from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel

#os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS


In [5]:
%%time
ratings = pd.read_csv(
    "test_sparse_matrix_dataset_ratings.dat",
    sep="::",
    engine="python",  # Because of 2-chars separators
    header=None,
    names=[Columns.User, Columns.Item, Columns.Weight, Columns.Datetime],
)
print(ratings.shape)
ratings.head()


(1000209, 4)
CPU times: total: 3.17 s
Wall time: 3.17 s


,user_id,item_id,weight,datetime
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
%%time
users = pd.read_csv(
    "test_sparse_matrix_dataset_users.dat",
    sep="::",
    engine="python",
    header=None,
    names=[Columns.User, "sex", "age", "occupation", "zip_code"],
)
print(users.shape)
users.head()

(6040, 5)
CPU times: total: 15.6 ms
Wall time: 17 ms


,user_id,sex,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
# берем только тех пользователей, которые есть в таблице рейтинга
users = users.loc[users["user_id"].isin(ratings["user_id"])].copy()

# Пример построения спарс-матрицы (разряженной матрицы)

In [9]:
# подготовка плоской таблицы с 3 фичами по пользователю
user_features_frames = []
for feature in ["sex", "age", "occupation"]:
    feature_frame = users.reindex(columns=["user_id", feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
    
user_features_frames

[        id value feature
 0        1     F     sex
 1        2     M     sex
 2        3     M     sex
 3        4     M     sex
 4        5     M     sex
 ...    ...   ...     ...
 6035  6036     F     sex
 6036  6037     F     sex
 6037  6038     F     sex
 6038  6039     F     sex
 6039  6040     M     sex
 
 [6040 rows x 3 columns],
         id  value feature
 0        1      1     age
 1        2     56     age
 2        3     25     age
 3        4     45     age
 4        5     25     age
 ...    ...    ...     ...
 6035  6036     25     age
 6036  6037     45     age
 6037  6038     56     age
 6038  6039     45     age
 6039  6040     25     age
 
 [6040 rows x 3 columns],
         id  value     feature
 0        1     10  occupation
 1        2     16  occupation
 2        3     15  occupation
 3        4      7  occupation
 4        5     20  occupation
 ...    ...    ...         ...
 6035  6036     15  occupation
 6036  6037      1  occupation
 6037  6038      1  occupatio

In [10]:
user_features = pd.concat(user_features_frames)
user_features

,id,value,feature
0,1,F,sex
1,2,M,sex
2,3,M,sex
3,4,M,sex
4,5,M,sex
...,...,...,...
6035,6036,15,occupation
6036,6037,1,occupation
6037,6038,1,occupation
6038,6039,0,occupation


In [11]:
%%time
# сборка датасета
sparse_features_dataset = Dataset.construct(
    ratings,
    user_features_df=user_features,  # our flatten dataframe
    cat_user_features=["sex", "age"], # these will be one-hot-encoded. All other features must be numerical already
    make_dense_user_features=False  # for `sparse` format
)

CPU times: total: 62.5 ms
Wall time: 56 ms


In [12]:
sparse_features_dataset

Dataset(user_id_map=IdMap(external_ids=array([   1,    2,    3, ..., 6038, 6039, 6040], dtype=int64)), item_id_map=IdMap(external_ids=array([1193,  661,  914, ..., 2845, 3607, 2909], dtype=int64)), interactions=Interactions(df=         user_id  item_id  weight                      datetime
0              0        0     5.0 1970-01-01 00:00:00.978300760
1              0        1     3.0 1970-01-01 00:00:00.978302109
2              0        2     3.0 1970-01-01 00:00:00.978301968
3              0        3     4.0 1970-01-01 00:00:00.978300275
4              0        4     5.0 1970-01-01 00:00:00.978824291
...          ...      ...     ...                           ...
1000204     6039      772     1.0 1970-01-01 00:00:00.956716541
1000205     6039     1106     5.0 1970-01-01 00:00:00.956704887
1000206     6039      365     5.0 1970-01-01 00:00:00.956704746
1000207     6039      152     4.0 1970-01-01 00:00:00.956715648
1000208     6039       26     4.0 1970-01-01 00:00:00.956715569

[100

In [16]:
sparse_features_dataset.user_id_map

IdMap(external_ids=array([   1,    2,    3, ..., 6038, 6039, 6040], dtype=int64))

In [17]:
sparse_features_dataset.item_id_map

IdMap(external_ids=array([1193,  661,  914, ..., 2845, 3607, 2909], dtype=int64))

In [18]:
sparse_features_dataset.interactions

Interactions(df=         user_id  item_id  weight                      datetime
0              0        0     5.0 1970-01-01 00:00:00.978300760
1              0        1     3.0 1970-01-01 00:00:00.978302109
2              0        2     3.0 1970-01-01 00:00:00.978301968
3              0        3     4.0 1970-01-01 00:00:00.978300275
4              0        4     5.0 1970-01-01 00:00:00.978824291
...          ...      ...     ...                           ...
1000204     6039      772     1.0 1970-01-01 00:00:00.956716541
1000205     6039     1106     5.0 1970-01-01 00:00:00.956704887
1000206     6039      365     5.0 1970-01-01 00:00:00.956704746
1000207     6039      152     4.0 1970-01-01 00:00:00.956715648
1000208     6039       26     4.0 1970-01-01 00:00:00.956715569

[1000209 rows x 4 columns])

В этом наборе данных пользовательские функции теперь хранятся в разреженном формате.    
    
В cat_user_features извлекаются все возможные значения, горячекодируются и сохраняются в разреженной матрице.    
    
Значения всех остальных признаков (прямых) хранятся в одной и той же разреженной матрице (один столбец для одного прямого признака). Здесь мы делаем «оккупацию» прямой функцией просто для быстрого примера хранения данных. На самом деле оно имеет категоричный характер.    
    
Строки разреженной матрицы соответствуют внутренним идентификаторам пользователей в наборе данных. Они идентичны номерам строк в матрице ui_csr, которая используется для обучения модели в большинстве рекомендательных моделей.    
    
Давайте заглянем внутрь набора данных, чтобы проверить, как хранятся данные.    

In [19]:
sparse_features_dataset.user_features.values

<6040x10 sparse matrix of type '<class 'numpy.float32'>'
	with 18120 stored elements in Compressed Sparse Row format>

In [20]:
sparse_features_dataset.user_features.names

(('occupation', '__is_direct_feature'),
 ('sex', 'F'),
 ('sex', 'M'),
 ('age', 1),
 ('age', 56),
 ('age', 25),
 ('age', 45),
 ('age', 50),
 ('age', 35),
 ('age', 18))

In [22]:
sparse_features_dataset.user_features.values[:5].toarray()

array([[10.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [16.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [15.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 7.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [20.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]], dtype=float32)

# Пример построения плотной матрицы

Теперь давайте создадим набор данных с плотными функциями.    
    
Нам нужен классический фрейм данных с одним столбцом для каждого объекта и одной строкой для каждого субъекта (пользователя или элемента).    
    
Важно: все значения функций должны быть числовыми.    
    
Важно: Вы должны установить функции для всех объектов (пользователей или элементов). Если у вас нет какой-либо функции для какого-либо пользователя (элемента), используйте любой метод (ноль, среднее значение и т. д.) для ее заполнения    

In [23]:
user_numeric_features = users[[Columns.User, "age", "occupation"]]
user_numeric_features.head()

,user_id,age,occupation
0,1,1,10
1,2,56,16
2,3,25,15
3,4,45,7
4,5,25,20


In [24]:
dense_features_dataset = Dataset.construct(
    ratings,
    user_features_df=user_numeric_features,
    make_dense_user_features=True  # for `dense` format
)

In [25]:
dense_features_dataset.user_features.names

('age', 'occupation')

In [26]:
dense_features_dataset.user_features.values[:5]

array([[ 1., 10.],
       [56., 16.],
       [25., 15.],
       [45.,  7.],
       [25., 20.]], dtype=float32)

# Передача функций в модели

Теперь мы можем просто подогнать модель, используя подготовленный набор данных. Для этого мы выбираем модели, которые поддерживают использование функций в обучении (например, iALS, LightFM, DSSM, PopularInCategory)

In [27]:
model = ImplicitALSWrapperModel(AlternatingLeastSquares(10, num_threads=32))
model.fit(dense_features_dataset)

C:\Programming\Anaconda\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
model = ImplicitALSWrapperModel(AlternatingLeastSquares(10, num_threads=32))
model.fit(sparse_features_dataset)

C:\Programming\Anaconda\lib\site-packages\rectools\dataset\features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


  0%|          | 0/1 [00:00<?, ?it/s]

# Заметки

* Если модели требуются функции в определенном формате, она автоматически преобразует их. Вот почему мы можем получить предупреждение, устанавливая iALS с редкими функциями. Модель в любом случае подойдет, только помните о возможных проблемах с памятью
* LightFM и DSSM предпочитают функции горячего кодирования. Поэтому хорошей идеей будет бинаризировать все прямые признаки и сделать их категориальными. Но вы также можете попробовать применить MinMaxScaler к прямым значениям.
* iALS хорошо работает как с прямыми, так и с категориальными признаками. Прямые функции могут быть MinMaxScaled.
* PopularInCategory требует небольшого количества функций и выбранной категории из-за ее природы.